In [1]:
import numpy as np
import pandas as pd

import hyperopt
from hyperopt import hp
from hyperopt.pyll.base import scope

from coremdlr.datasets import DepthSequenceGenerator, FaciesDataset
from coremdlr.networks import rmlp

Using TensorFlow backend.


In [2]:
from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.models import FeaturePredictor, LambdaModel

In [3]:
fdset = FaciesDataset(["205-21b-3", "204-19-6", "204-24a-6"],
                    test_wells=["204-20-6a"],
                    features=["logs"],
                    logs_args={'which_logs': ['GR','DENS','NEUT','PEF','RDEP','RSHAL', 'DTC']},
                    label_resolution=32)

fdset.load_or_generate_data()

Loading Well:  205-21b-3  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('logs', (3842, 7))]
Loading Well:  204-19-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Feature shapes:  [('depth', (1884,)), ('top', (1884,)), ('base', (1884,)), ('logs', (1884, 7))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 7))]
Loading Well:  204-20-6a  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


ValueError: b'os' is not in list

In [4]:
train_generator = DepthSequenceGenerator(fdset.X_train['logs'], fdset.y_train, 256, step_size=64, batch_size=8)
test_generator = DepthSequenceGenerator(fdset.X_test['logs'], fdset.y_test, 256, step_size=64, batch_size=8)

RLMP_SEARCH_SPACE = {
    'layer_type' : hp.choice('layer_type', ['lstm', 'gru']),
    'hidden1' : scope.int(hp.quniform('hidden1', 7, 64, 1)),
    'hidden2' : scope.int(hp.quniform('hidden2', 7, 64, 1)),
    'dropout_rate' : hp.uniform('dropout_rate', 0.0, 0.95),
}

results_df = pd.DataFrame(columns=list(RLMP_SEARCH_SPACE.keys()) + ['best_epoch', 'val_loss', 'val_acc'])

def train_rmlp(model_config):
    print(model_config)
    rnet = rmlp((256, 7), 4, **model_config)
    print(rnet.summary())
    
    rnet.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    hist = rnet.fit_generator(train_generator, epochs=10, validation_data=test_generator, shuffle=False)
    
    new_row = model_config
    new_row['best_epoch'] = np.argmin(hist.history['val_loss'])
    new_row['val_loss'] = min(hist.history['val_loss'])
    new_row['val_acc'] = min(hist.history['val_acc'])
    results_df = results_df.append(new_row, ignore_index=True)
    
    return new_row['val_loss']

AttributeError: 'FaciesDataset' object has no attribute 'X_test'

In [14]:
best_params = hyperopt.fmin(
    fn=train_rmlp,
    space=RLMP_SEARCH_SPACE,
    algo=hyperopt.rand.suggest,
    max_evals=100
)
print(best_params)

{'dropout_rate': 0.04497765997112445, 'hidden1': 54, 'hidden2': 60, 'layer_type': 'lstm'}


TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [6]:
results_df.to_csv('./rlmp_2layer_results.csv')

In [16]:
train_generator = DepthwiseSequenceGenerator(fdset.X_train['logs'], fdset.y_train, 256, step_size=64, batch_size=8)
test_generator = DepthwiseSequenceGenerator(fdset.X_test['logs'], fdset.y_test, 256, step_size=64, batch_size=8)

RLMP_SEARCH_SPACE = {
    'layer_type' : hp.choice('layer_type', ['lstm', 'gru']),
    'hidden1' : scope.int(hp.quniform('hidden1', 7, 64, 1)),
    'hidden2' : scope.int(hp.quniform('hidden2', 7, 64, 1)),
    'dropout_rate' : hp.uniform('dropout_rate', 0.0, 0.95),
}

results_df = pd.DataFrame(columns=list(RLMP_SEARCH_SPACE.keys()) + ['best_epoch', 'val_loss', 'val_acc'])

def train_rmlp(model_config):
    print(model_config)
    rnet = rmlp((256, 7), 4, **model_config)
    print(rnet.summary())
    
    rnet.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    hist = rnet.fit_generator(train_generator, epochs=10, validation_data=test_generator, shuffle=False)
    
    new_row = model_config
    new_row['best_epoch'] = np.argmin(hist.history['val_loss'])
    new_row['val_loss'] = min(hist.history['val_loss'])
    new_row['val_acc'] = min(hist.history['val_acc'])
    results_df = results_df.append(new_row, ignore_index=True)
    
    return new_row['val_loss']

NameError: name 'DepthwiseSequenceGenerator' is not defined

In [17]:
best_params = hyperopt.fmin(
    fn=train_rmlp,
    space=RLMP_SEARCH_SPACE,
    algo=hyperopt.rand.suggest,
    max_evals=2
)
print(best_params)

{'dropout_rate': 0.5384616633013634, 'hidden1': 21, 'hidden2': 46, 'layer_type': 'gru'}


TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'